<h1>目录</h1>

1. [超分辨的概念与应用](#超分辨的概念与应用)<br>

2. [超分辨的典型模型](#超分辨的典型模型)<br>
   [1. 前上采样（pre-upsampling）](#1.-前上采样（pre-upsampling）)<br>
   [2. 后上采样（post-upsampling）](#2.-后上采样（post-upsampling）)<br>

# 超分辨的概念与应用

我们常说的图像分辨率指的是图像长边像素数与图像短边像素数的乘积，比如iPhoneX手机拍摄照片的分辨率为 **4032px×3024px**，为1200万像素。

显然，越高的分辨率能获得更清晰的成像。与之同时，分辨率越高也意味着更大的存储空间，对于空间非常有限的移动设备来说，需要考虑分辨率与存储空间的平衡。

图像超分，就是要从低分辨率的图像恢复为高分辨率的图像，它在日常的图像和视频存储与浏览中都有广泛的应用。

10年前手机中320px×240px分辨率的图像是主流，其视觉美感相对如今随处可见的4K分辨率来说是无法比拟的。我们可以使用超分技术来恢复当年拍摄的低分辨率图像，如下图是一个典型案例。手机图片浏览中也有超分算法的努力，即同一张图片在不同手机上的显示效果不一样，是因为显示分辨率越高的手机可以使用更清晰的分辨率进行展示。




<center>
    <img style="border-radius: 0.3125em;
    box-shadow: 0 8px 8px 0 rgba(34,36,38,.12),0 5px 10px 0 rgba(34,36,38,.08);" 
    src="https://tianchi-public.oss-cn-hangzhou.aliyuncs.com/public/files/forum/162037459533567181620374592872.jpeg">
    <br>
    <div style="color:orange; border-bottom: 1px solid #d9d9d9;
    display: inline-block;
    color: #999;
    padding: 2px;">老旧照片超分，左图为原图，右图为调整后的图</div>
</center>





# 超分辨的典型模型

近年来CNN等深度学习模型在图像超分任务中取得了非常大的进展，使得超分算法得以真正在产品中落地，目前根据上采样（upsampling）在网络结构中的位置和使用方式不同，最主流的超分网络架构有两种。

## 1. 前上采样（pre-upsampling）
> **即在网络一开始的时候就完成上采样过程。**

Chao Dong等人提出的SRCNN方法是最早期的尝试，流程示意图如下图所示:

<center>
    <img style="border-radius: 0.3125em;
    box-shadow: 0 8px 8px 0 rgba(34,36,38,.12),0 5px 10px 0 rgba(34,36,38,.08);" 
    src="https://tianchi-public.oss-cn-hangzhou.aliyuncs.com/public/files/forum/162037463187544471620374630540.png">
    <br>
    <div style="color:orange; border-bottom: 1px solid #d9d9d9;
    display: inline-block;
    color: #999;
    padding: 2px;">基于前上采样的SRCNN模型</div>
</center>



SRCNN框架首先使用双线性插值等上采样方法进行初始化，得到想要恢复的分辨率，这一步也可以使用反卷积来完成。
然后使用卷积层对输入的局部图像块进行特征提取，得到一系列特征图，这相当于完成了稀疏编码中重叠的图像块的构建，这一步骤可以表达如下:

$$\boldsymbol{F}_{\mathbf{1}}(\mathbf{Y})=\max \left(\boldsymbol{W}_{\mathbf{1}} * \boldsymbol{Y}+\boldsymbol{B}_{\mathbf{1}}\right)$$

其中 $W_1$ 和 $B_1$ 分别表示卷积核和偏置，$*$ 表示卷积操作，$Y$ 就是输入。$W_1$ 的尺寸为 $c \times n_{1} \times f_{1} \times f_{1}$ ,其中 $c$ 就是输入图的通道数量，$n_1$ 是输出特征通道数，$f_{1} \times f_{1}$是卷积核大小。

早期的超分算法常常只对亮度通道进行超分，颜色通道则进行双线性上采样。SRCNN算法则同时对RGB通道进行了学习，因为这三个通道之间存在较强的灰度耦合性。

接着，使用 1×1 卷积进行维度变换，即将 $n_1$ 个特征通道转换为 $n_2$ 个特征通道，这就是相当于稀疏编码中低分辨率字典到高分辨率字典的映射，这一步骤可以表达如下：

$$\boldsymbol{F}_{2}(\mathbf{Y})=\max \left(\boldsymbol{W}_{2} * \boldsymbol{F}_{\mathbf{1}}(\mathbf{Y})+\boldsymbol{B}_{\mathbf{2}}\right)$$

$W_2$ 的尺寸为 $n_1×n_2×f_2 ×f_2$ ,其中 $n_1$ 就是输入特征的通道数量，$n_2$ 是输出特征通道数，$f_2×f_2$ 是卷积核大小，实际上 $f_2=1$。

最后就是将高分辨率的图像块重新拼接成完整的图像，这一步骤可以表达如下：

$$
\boldsymbol{F}_{3}(\mathbf{Y})=\max \left(\boldsymbol{W}_{3} * \boldsymbol{F}_{2}(\mathbf{Y})+\boldsymbol{B}_{3}\right)
$$

$W_3$ 的尺寸为 $n_2×c×f_3 ×f_3$ ,其中 $n_2$ 就是输入特征的通道数量，c 是输出特征通道数，它等于输入图像的通道数，$f_3×f_3$ 是卷积核大小。

当 $f_2=1$ 时，对于输出图像中的每一个像素，它在原图中的感受野大小为$(f_3+f_1-1)^2$，一个典型的设定是$f_1=9$，$f_3=5$，此时输出像素与输出的 $13^2=169$ 个像素有关，相比于传统方法具有较大的感受野，因此SRCNN具有较大的优势。

SRCNN框架可以适用于任意分辨率的提升，因为在输入网络之前，上采样过程已经对输出分辨率做了初始化，所以CNN模型要学习的是由粗到精的改进，学习过程比较简单。不过由于整个网络在高分辨率空间进行计算，因此计算量大，而且噪声容易被放大。

SRCNN的整个流程与稀疏编码方法相同，因此它也被看作是使用CNN实现了稀疏编码的方案。


## 2. 后上采样（post-upsampling）
> **即在网络的最后才开始进行上采样。**

在前上采样框架中首先使用反卷积来完成上采样是一种很自然的操作，但是它计算复杂度较大，因此SRCNN的作者后来将该上采样过程放置在网络最后端，通过一个反卷积来学习该上采样过程，将其命名为FSRCNN框架。而Twitter图片与视频压缩研究组则采用了与反卷积完全不同的上采样思路，提出了ESPCN模型，其中核心思想是亚像素卷积(sub-pixel convolution)，完整流程示意图如下：

<center>
    <img style="border-radius: 0.3125em;
    box-shadow: 0 8px 8px 0 rgba(34,36,38,.12),0 5px 10px 0 rgba(34,36,38,.08);" 
    src="https://tianchi-public.oss-cn-hangzhou.aliyuncs.com/public/files/forum/162037466438645871620374661118.png">
    <br>
    <div style="color:orange; border-bottom: 1px solid #d9d9d9;
    display: inline-block;
    color: #999;
    padding: 2px;">基于亚像素卷积的后上采样ESPCN模型</div>
</center>


对于维度为H×W×C的图像，标准反卷积操作输出的特征图维度为rH×rW×C，其中r就是需要放大的倍数，而从上图可以看出，亚像素卷积层的输出特征图维度为H×W×C×r2，即特征图与输入图片的尺寸保持一致，但是通道数被扩充为原来的r2倍，然后再进行重新排列得到高分辨率的结果。

整个流程因为使用了更小的图像输入，从而可以使用更小的卷积核获取较大的感受野，这既使得输入图片中邻域像素点的信息得到有效利用，还避免了计算复杂度的增加，是一种将空间上采样问题转换为通道上采样问题的思路。

相比于前上采样中在开始就进行单一的一次上采样，后采样策略能更好地利用模型的表达能力，学习更加复杂的低分辩率到高分辨率的转换，因此ESPCN模型被验证为更加有效，后续的超分模型基本沿用了该思路。
